### AWS Boto3 Assignment

In this assignment, you'll be covering the most common AWS Boto3 examples required. Refer the official [Boto3 documentation](boto3.readthedocs.io) if you get stuck somewhere.

### Task 1: Write a function called myKeyPair() to generate a keypair for a session:

- Accepts the following parameters:
    * keypair_tag, name of the key pair
    * region
    * profile_name, the AWS Profile you want to use

- Should return
    * keypair (same as the keypair_tag you sent)

In [4]:
def myKeyPair(keypair_tag, region, profile_name):
    
    import boto3
    
    # EC2 Resource creation
    sess = boto3.Session(profile_name=profile_name)
    
    ec2 = sess.resource('ec2')
    response = ec2.create_key_pair(KeyName=keypair_tag)
    
    outfile = open('/home/pratos/.ssh/file1.pem','w')
    KeyPairOut = str(response.key_material)
    outfile.write(KeyPairOut)
    
    return keypair_tag

In [5]:
keypair = myKeyPair('demo', 'us-east-1', 'default')

In [6]:
keypair

'demo'

### Task 2: Write a function called myInstanceCreator():

- Accepts the following parameters:
    * profile_name
    * region
    * min_count, minumum number of instances
    * max_count, maximum number of instances
    * keypair
    * instance type
    * disksize
    * ami-id

- Should return
    * instance ids

In [11]:
def myInstanceCreator(profile_name, region, min_count, max_count, keypair, instance_type, disksize, ami_id):
    import boto3
    
    # EC2 Resource creation
    sess = boto3.Session(profile_name=profile_name)
    
    ec2 = sess.resource('ec2')
    print("Session to access EC2 resources has been created...")
    
    print("Creating the EC2 instances...")
    dl_instance = ec2.create_instances(ImageId=ami_id, 
                                            MinCount=min_count, #Minimum number of instances 
                                            MaxCount=max_count, #Maximum number of instances 
                                            KeyName=keypair, 
                                            InstanceType=instance_type,
                                            BlockDeviceMappings=[{"DeviceName": "/dev/sda1","Ebs" : \
                                                                  { "VolumeSize" : disksize, \
                                                                    "VolumeType" : "standard"}}],
                                          ) 
    
    print
    return(dl_instance)

In [12]:
instance_ids = myInstanceCreator(profile_name='default', region='us-east-1', min_count=1, max_count=1, \
                                keypair=keypair, instance_type='t2.micro', disksize=20, ami_id='ami-e47723f2')

Session to access EC2 resources has been created...
Creating the EC2 instances...


In [17]:
instance_ids

[ec2.Instance(id='i-043a2363065353b33')]

### Task 3: List the number of EC2 instances in a region by creating function, myInstanceList()

- Accepts the following parameters:
    * region
    * profile_name
    
- Should return
    * list of instance ids

In [21]:
def myInstanceList(profile_name):
    import boto3
    
    # EC2 Resource creation
    sess = boto3.Session(profile_name=profile_name)
    
    ec2 = sess.resource('ec2')
    instances = ec2.instances.filter(
    Filters=[{'Name': 'instance-state-name', 'Values': ['running']}])
    
    instance_ids = []
    for instance in instances:
        instance_ids.append(instance.id)
    
    return instance_ids

In [22]:
instance_ids = myInstanceList('default')

In [23]:
instance_ids

['i-08b0593cb0cb54f5e']

### Task 3: Write a function called myInstanceOptions():

- Accepts the following parameters:
    * list of instance ids
    * action, the operation to be performed on the instance: "stop", "start", "terminate", "reboot"
 
- Should return
    * HTTP Response from AWS

In [31]:
def myInstanceOptions(profile_name, instance_ids, action):
    import boto3
    
    # EC2 Resource creation
    sess = boto3.Session(profile_name=profile_name)
    
    ec2_client = sess.client('ec2')
    final_response = None
    
    if action.lower() == "start":
        try:
            print("Starting the instances again...")
            response = ec2_client.start_instances(InstanceIds=instance_ids, DryRun=False)
            final_response = response
        except ClientError as e:
            print(e)
    elif action.lower() == "stop":
        try:
            print("Stopping the instances...")
            response = ec2_client.stop_instances(InstanceIds=instance_ids, DryRun=False)
            final_response = response
        except ClientError as e:
            print(e)
    elif action.lower() == "terminate":
        try:
            print("Terminating the instances...")
            response = ec2_client.terminate_instances(InstanceIds=instance_ids, DryRun=False)
            final_response = response
        except ClientError as e:
            print("Error: {}".format(e))
    elif action.lower() == "reboot":
        try:
            print("Rebooting the instances...")
            response = ec2_client.reboot_instances(InstanceIds=instance_ids, DryRun=False)
            final_response = response
        except ClientError as e:
            print("Error: {}".format(e))
            
    return final_response

In [32]:
myInstanceOptions('default', instance_ids, "Stop")

Starting the instances again...


{'ResponseMetadata': {'HTTPHeaders': {'content-length': '540',
   'content-type': 'text/xml;charset=UTF-8',
   'date': 'Sun, 10 Sep 2017 13:32:30 GMT',
   'server': 'AmazonEC2'},
  'HTTPStatusCode': 200,
  'RequestId': '71176c44-7f89-4bd8-a16d-9ec431ffda89',
  'RetryAttempts': 0},
 u'StoppingInstances': [{u'CurrentState': {u'Code': 64, u'Name': 'stopping'},
   u'InstanceId': 'i-08b0593cb0cb54f5e',
   u'PreviousState': {u'Code': 16, u'Name': 'running'}}]}

### Task 4: Creating or Delete an S3 bucket using function, myManageBucket():

- Accepts the following parameters:
    * profile_name,  the AWS Profile you want to use
    * bucket_name
    * action, the operation to be performed: "create", "delete"
 
- Should return
    * HTTP Response from AWS

In [33]:
def myManageBucket(profile_name, bucket_name, action):
    import boto3
    
    # EC2 Resource creation
    sess = boto3.Session(profile_name=profile_name)
    
    response = None
    
    if action.lower() == "create":
        s3 = sess.client('s3')
        
        print("Creating an S3 bucket...")
        response = s3.create_bucket(Bucket=bucket_name)
    elif action.lower() == "delete":
        s3 = sess.resource('s3')
        
        print("Deleting the bucket")
        for key in bucket.objects.all():
            key.delete()
            print("{} deleted...".format(key))

        response = bucket.delete()
    
    return (response)

In [40]:
myManageBucket('default', 'greyatom-demo', 'create')

Creating an S3 bucket...


{u'Location': '/greyatom-demo',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Sun, 10 Sep 2017 14:01:38 GMT',
   'location': '/greyatom-demo',
   'server': 'AmazonS3',
   'x-amz-id-2': 'gBh/sjw3JKDKa6QO5gSvWclCnC5N+2Lnj80AKKZMIO5JYxdWOBjDU1fhappWQr9HVrtNrvd9gy4=',
   'x-amz-request-id': '9660C3613FEFFCE1'},
  'HTTPStatusCode': 200,
  'HostId': 'gBh/sjw3JKDKa6QO5gSvWclCnC5N+2Lnj80AKKZMIO5JYxdWOBjDU1fhappWQr9HVrtNrvd9gy4=',
  'RequestId': '9660C3613FEFFCE1',
  'RetryAttempts': 0}}

### Task 5: Performing add, delete on S3, myBucketOperations():

- Accepts the following parameters:
    * profile_name,  the AWS Profile you want to use
    * bucket_name
    * list_of_files, it's default value is __None__
    * action, the operation to be performed: "upload", "download"
 
- Should return
    * List of the files present in the bucket after upload

In [74]:
def myBucketOperations(profile_name, bucket_name, action, list_of_files=None):
    import re
    import boto3
    
    # EC2 Resource creation
    sess = boto3.Session(profile_name=profile_name)
    
    response = []
    
    if action.lower() == "download":
        bucket_name = bucket_name
        key = list_of_files

        s3 = sess.client('s3')

        for filename in list_of_files:
            filename_final = re.split(r'/', filename)
            print("Downloading file: {}".format(filename_final[len(filename_final)-1]))
            s3.download_file(bucket_name, filename_final[len(filename_final)-1]), filename
            
            s3_resource = boto3.resource('s3')

            my_bucket = s3_resource.Bucket(bucket_name)

            for obj in my_bucket.objects.all():
                response.append(obj.key)
        
    elif action.lower() == "upload":
        bucket_name = bucket_name
        key = list_of_files

        s3 = sess.client('s3')
        
        for filename in list_of_files:            
            filename_final = re.split(r'/', filename)
            print("Uploading file: {}".format(filename_final[len(filename_final)-1]))
            response.append(s3.upload_file(filename, bucket_name, filename_final[len(filename_final)-1]))
        
        s3_resource = boto3.resource('s3')

        my_bucket = s3_resource.Bucket(bucket_name)

        for obj in my_bucket.objects.all():
            response.append(obj.key)
            
    return(response)

In [75]:
import glob
list_of_files = glob.glob('/home/pratos/greyatom_final/day29_machine_learning_in_production_one/data/*.csv')

In [76]:
list_of_files

['/home/pratos/greyatom_final/day29_machine_learning_in_production_one/data/energy_efficiency.csv',
 '/home/pratos/greyatom_final/day29_machine_learning_in_production_one/data/boston.csv']

In [77]:
myBucketOperations('default', 'greyatom-demo', "upload", list_of_files)

Uploading file: energy_efficiency.csv
Uploading file: boston.csv


[None, None, u'boston.csv', u'energy_efficiency.csv']